In [75]:
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [76]:
import tensorflow as tf
import datetime
import import_ipynb
from tensorboard.plugins.hparams import api as hp
from dataloader import get_LFW_X_df, get_y_df, data_prep

In [77]:
rm -rf ./logs/

In [78]:
csv_filepath = "../../data/csv/plant_data.csv"

In [79]:
X = get_LFW_X_df(csv_filepath)
y = get_y_df(csv_filepath, 'LFW_g')

In [80]:
HP_NUM_UNITS = hp.HParam("num_units", hp.Discrete([16, 32, 64, 128]))
HP_DROPOUT = hp.HParam("dropout", hp.RealInterval(0.1, 0.2))
HP_OPTIMIZER = hp.HParam("optimizer", hp.Discrete(['adam', 'sgd']))
HP_ACTIVATION = hp.HParam("activation", hp.Discrete(['linear', 'relu', 'sigmoid']))

METRIC_ACCURACY = 'loss'

In [81]:
with tf.summary.create_file_writer('../../logs/hyperparam_tuning/repression/LFW').as_default():
    hp.hparams_config(
        hparams=[HP_NUM_UNITS, HP_DROPOUT, HP_OPTIMIZER, HP_ACTIVATION],
        metrics=[hp.Metric(METRIC_ACCURACY, display_name="Test Accuracy")]
    )

In [82]:
X_train, X_test, y_train, y_test = data_prep(X, y)

In [83]:
def LFW_model(hparams):
        model = tf.keras.models.Sequential([
            tf.keras.layers.Dense(hparams[HP_NUM_UNITS], hparams[HP_ACTIVATION], name='layers_input', input_shape = (11,),),
            tf.keras.layers.Dropout(hparams[HP_DROPOUT]),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.Dense(hparams[HP_NUM_UNITS], hparams[HP_ACTIVATION], name='layers_dense2'),
            tf.keras.layers.Dropout(hparams[HP_DROPOUT]),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.Dense(1, name='layers_dense3')
        ])

        model.compile (
            optimizer=hparams[HP_OPTIMIZER],
            loss='mae',
        )

        model.fit (
            X_train,
            y_train,
            epochs=100,
            validation_split=0.2,
            callbacks=[
                tf.keras.callbacks.TensorBoard(log_dir=log_dir),
                hp.KerasCallback(log_dir, hparams)
            ]
        )

        accuracy = model.evaluate(X_test, y_test)
        return accuracy

        

In [84]:
log_dir = "../../logs/hyperparam_tuning/regression/LFW/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [85]:
def run_model(run_dir, hparams):
    with tf.summary.create_file_writer(run_dir).as_default():
        hp.hparams(hparams)
        accuracy = LFW_model(hparams)
        tf.summary.scalar(METRIC_ACCURACY, accuracy, step=1)

In [86]:
session_num = 0

for num_units in HP_NUM_UNITS.domain.values:
    for droput in (HP_DROPOUT.domain.min_value, HP_DROPOUT.domain.max_value):
        for activation in HP_ACTIVATION.domain.values:
            for optimizer in HP_OPTIMIZER.domain.values:
                hparams = {
                    HP_ACTIVATION: activation,
                    HP_DROPOUT: droput,
                    HP_NUM_UNITS: num_units,
                    HP_OPTIMIZER: optimizer
                }

                run_name = "run-%d" % session_num
                print('--- Starting trial: %s' % run_name)
                print({h.name: hparams[h] for h in hparams})
                run_model(log_dir + run_name, hparams)
                session_num += 1

--- Starting trial: run-0
{'activation': 'linear', 'dropout': 0.1, 'num_units': 16, 'optimizer': 'adam'}
Epoch 1/100
11/11 [==============================] - 2s 26ms/step - loss: 62.5025 - val_loss: 59.6822
Epoch 2/100
11/11 [==============================] - 0s 9ms/step - loss: 62.4364 - val_loss: 59.6261
Epoch 3/100
11/11 [==============================] - 0s 8ms/step - loss: 62.3651 - val_loss: 59.5677
Epoch 4/100
11/11 [==============================] - 0s 9ms/step - loss: 62.2874 - val_loss: 59.4982
Epoch 5/100
11/11 [==============================] - 0s 9ms/step - loss: 62.2026 - val_loss: 59.4326
Epoch 6/100
11/11 [==============================] - 0s 8ms/step - loss: 62.1100 - val_loss: 59.3551
Epoch 7/100
11/11 [==============================] - 0s 7ms/step - loss: 62.0093 - val_loss: 59.2640
Epoch 8/100
11/11 [==============================] - 0s 8ms/step - loss: 61.9001 - val_loss: 59.1681
Epoch 9/100
11/11 [==============================] - 0s 9ms/step - loss: 61.7821 - val